In [2]:
# Since my computer gpu is really old-fashioned, I used a library about ignore warnings
import warnings
warnings.filterwarnings(action='once')

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [15]:
# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units


# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [16]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))


Epoch  1, Batch   1 - Loss: 50704.3672 Validation Accuracy: 0.183594
Epoch  1, Batch   2 - Loss: 35590.1211 Validation Accuracy: 0.187500
Epoch  1, Batch   3 - Loss: 31317.2129 Validation Accuracy: 0.179688
Epoch  1, Batch   4 - Loss: 30672.3945 Validation Accuracy: 0.199219
Epoch  1, Batch   5 - Loss: 23368.0156 Validation Accuracy: 0.203125
Epoch  1, Batch   6 - Loss: 25165.0117 Validation Accuracy: 0.199219
Epoch  1, Batch   7 - Loss: 20871.6992 Validation Accuracy: 0.191406
Epoch  1, Batch   8 - Loss: 25595.7754 Validation Accuracy: 0.175781
Epoch  1, Batch   9 - Loss: 22365.2227 Validation Accuracy: 0.167969
Epoch  1, Batch  10 - Loss: 21276.0371 Validation Accuracy: 0.183594
Epoch  1, Batch  11 - Loss: 21944.6445 Validation Accuracy: 0.207031
Epoch  1, Batch  12 - Loss: 22129.5879 Validation Accuracy: 0.230469
Epoch  1, Batch  13 - Loss: 17101.2129 Validation Accuracy: 0.257812
Epoch  1, Batch  14 - Loss: 15840.1094 Validation Accuracy: 0.261719
Epoch  1, Batch  15 - Loss: 16370.

Epoch  1, Batch 120 - Loss:  3260.4375 Validation Accuracy: 0.636719
Epoch  1, Batch 121 - Loss:  3590.4985 Validation Accuracy: 0.660156
Epoch  1, Batch 122 - Loss:  3941.5862 Validation Accuracy: 0.660156
Epoch  1, Batch 123 - Loss:  3570.6416 Validation Accuracy: 0.664062
Epoch  1, Batch 124 - Loss:  3468.8210 Validation Accuracy: 0.660156
Epoch  1, Batch 125 - Loss:  4709.0820 Validation Accuracy: 0.656250
Epoch  1, Batch 126 - Loss:  3256.1929 Validation Accuracy: 0.667969
Epoch  1, Batch 127 - Loss:  3908.8801 Validation Accuracy: 0.671875
Epoch  1, Batch 128 - Loss:  4575.6074 Validation Accuracy: 0.675781
Epoch  1, Batch 129 - Loss:  4725.7578 Validation Accuracy: 0.683594
Epoch  1, Batch 130 - Loss:  4507.2578 Validation Accuracy: 0.683594
Epoch  1, Batch 131 - Loss:  4203.4971 Validation Accuracy: 0.675781
Epoch  1, Batch 132 - Loss:  4081.1519 Validation Accuracy: 0.679688
Epoch  1, Batch 133 - Loss:  3381.4871 Validation Accuracy: 0.671875
Epoch  1, Batch 134 - Loss:  3216.

Epoch  1, Batch 239 - Loss:  2261.5283 Validation Accuracy: 0.750000
Epoch  1, Batch 240 - Loss:  2145.7354 Validation Accuracy: 0.753906
Epoch  1, Batch 241 - Loss:  2400.1855 Validation Accuracy: 0.761719
Epoch  1, Batch 242 - Loss:  3081.4595 Validation Accuracy: 0.761719
Epoch  1, Batch 243 - Loss:  2838.1460 Validation Accuracy: 0.761719
Epoch  1, Batch 244 - Loss:  2996.2339 Validation Accuracy: 0.765625
Epoch  1, Batch 245 - Loss:  2094.7156 Validation Accuracy: 0.765625
Epoch  1, Batch 246 - Loss:  2211.4529 Validation Accuracy: 0.761719
Epoch  1, Batch 247 - Loss:  2129.4033 Validation Accuracy: 0.761719
Epoch  1, Batch 248 - Loss:  2160.3853 Validation Accuracy: 0.761719
Epoch  1, Batch 249 - Loss:  1446.7554 Validation Accuracy: 0.757812
Epoch  1, Batch 250 - Loss:  2311.9888 Validation Accuracy: 0.761719
Epoch  1, Batch 251 - Loss:  1849.9363 Validation Accuracy: 0.761719
Epoch  1, Batch 252 - Loss:  1773.2444 Validation Accuracy: 0.761719
Epoch  1, Batch 253 - Loss:  2111.

Epoch  1, Batch 358 - Loss:  1654.4880 Validation Accuracy: 0.781250
Epoch  1, Batch 359 - Loss:  1460.6946 Validation Accuracy: 0.789062
Epoch  1, Batch 360 - Loss:  2112.2075 Validation Accuracy: 0.781250
Epoch  1, Batch 361 - Loss:  2569.8193 Validation Accuracy: 0.785156
Epoch  1, Batch 362 - Loss:  2496.7385 Validation Accuracy: 0.785156
Epoch  1, Batch 363 - Loss:  1669.7925 Validation Accuracy: 0.785156
Epoch  1, Batch 364 - Loss:  1926.0480 Validation Accuracy: 0.792969
Epoch  1, Batch 365 - Loss:  2259.0728 Validation Accuracy: 0.789062
Epoch  1, Batch 366 - Loss:  2015.9385 Validation Accuracy: 0.785156
Epoch  1, Batch 367 - Loss:  2135.0764 Validation Accuracy: 0.789062
Epoch  1, Batch 368 - Loss:  1431.4280 Validation Accuracy: 0.789062
Epoch  1, Batch 369 - Loss:  1984.9678 Validation Accuracy: 0.789062
Epoch  1, Batch 370 - Loss:  2042.5867 Validation Accuracy: 0.792969
Epoch  1, Batch 371 - Loss:  1296.1201 Validation Accuracy: 0.792969
Epoch  1, Batch 372 - Loss:  1447.

Epoch  2, Batch  50 - Loss:   956.6095 Validation Accuracy: 0.828125
Epoch  2, Batch  51 - Loss:  1273.7563 Validation Accuracy: 0.828125
Epoch  2, Batch  52 - Loss:  1787.8226 Validation Accuracy: 0.828125
Epoch  2, Batch  53 - Loss:  1573.1838 Validation Accuracy: 0.824219
Epoch  2, Batch  54 - Loss:  1282.1077 Validation Accuracy: 0.828125
Epoch  2, Batch  55 - Loss:  1189.5294 Validation Accuracy: 0.828125
Epoch  2, Batch  56 - Loss:  1661.9014 Validation Accuracy: 0.828125
Epoch  2, Batch  57 - Loss:  1662.5338 Validation Accuracy: 0.828125
Epoch  2, Batch  58 - Loss:  1191.9001 Validation Accuracy: 0.828125
Epoch  2, Batch  59 - Loss:  1376.2032 Validation Accuracy: 0.828125
Epoch  2, Batch  60 - Loss:  1340.5304 Validation Accuracy: 0.824219
Epoch  2, Batch  61 - Loss:  1009.6254 Validation Accuracy: 0.828125
Epoch  2, Batch  62 - Loss:  1364.3582 Validation Accuracy: 0.824219
Epoch  2, Batch  63 - Loss:   834.1294 Validation Accuracy: 0.824219
Epoch  2, Batch  64 - Loss:   919.

...

Epoch  6, Batch 376 - Loss:   366.5304 Validation Accuracy: 0.863281
Epoch  6, Batch 377 - Loss:   359.7718 Validation Accuracy: 0.863281
Epoch  6, Batch 378 - Loss:   514.4476 Validation Accuracy: 0.863281
Epoch  6, Batch 379 - Loss:   417.7847 Validation Accuracy: 0.863281
Epoch  6, Batch 380 - Loss:   411.6027 Validation Accuracy: 0.863281
Epoch  6, Batch 381 - Loss:   555.1716 Validation Accuracy: 0.863281
Epoch  6, Batch 382 - Loss:   438.3192 Validation Accuracy: 0.863281
Epoch  6, Batch 383 - Loss:   465.7579 Validation Accuracy: 0.859375
Epoch  6, Batch 384 - Loss:   429.0485 Validation Accuracy: 0.863281
Epoch  6, Batch 385 - Loss:   414.0128 Validation Accuracy: 0.863281
Epoch  6, Batch 386 - Loss:   252.3283 Validation Accuracy: 0.863281
Epoch  6, Batch 387 - Loss:   260.7875 Validation Accuracy: 0.863281
Epoch  6, Batch 388 - Loss:   641.8347 Validation Accuracy: 0.863281
Epoch  6, Batch 389 - Loss:   434.0760 Validation Accuracy: 0.863281
Epoch  6, Batch 390 - Loss:   476.

Epoch  7, Batch  65 - Loss:   360.9256 Validation Accuracy: 0.867188
Epoch  7, Batch  66 - Loss:   353.6378 Validation Accuracy: 0.863281
Epoch  7, Batch  67 - Loss:   190.1570 Validation Accuracy: 0.863281
Epoch  7, Batch  68 - Loss:   476.3610 Validation Accuracy: 0.863281
Epoch  7, Batch  69 - Loss:   295.4626 Validation Accuracy: 0.863281
Epoch  7, Batch  70 - Loss:   382.9763 Validation Accuracy: 0.863281
Epoch  7, Batch  71 - Loss:   470.3063 Validation Accuracy: 0.863281
Epoch  7, Batch  72 - Loss:   421.8744 Validation Accuracy: 0.863281
Epoch  7, Batch  73 - Loss:   584.1645 Validation Accuracy: 0.863281
Epoch  7, Batch  74 - Loss:   665.2177 Validation Accuracy: 0.863281
Epoch  7, Batch  75 - Loss:   156.9586 Validation Accuracy: 0.867188
Epoch  7, Batch  76 - Loss:   328.2239 Validation Accuracy: 0.863281
Epoch  7, Batch  77 - Loss:   347.2785 Validation Accuracy: 0.863281
Epoch  7, Batch  78 - Loss:   261.0749 Validation Accuracy: 0.863281
Epoch  7, Batch  79 - Loss:   212.

Epoch  7, Batch 183 - Loss:   331.7614 Validation Accuracy: 0.867188
Epoch  7, Batch 184 - Loss:   216.6164 Validation Accuracy: 0.867188
Epoch  7, Batch 185 - Loss:   468.2078 Validation Accuracy: 0.863281
Epoch  7, Batch 186 - Loss:   244.7687 Validation Accuracy: 0.863281
Epoch  7, Batch 187 - Loss:   293.9090 Validation Accuracy: 0.863281
Epoch  7, Batch 188 - Loss:   327.4628 Validation Accuracy: 0.863281
Epoch  7, Batch 189 - Loss:   529.7180 Validation Accuracy: 0.859375
Epoch  7, Batch 190 - Loss:   457.3577 Validation Accuracy: 0.859375
Epoch  7, Batch 191 - Loss:   250.8528 Validation Accuracy: 0.863281
Epoch  7, Batch 192 - Loss:   235.0807 Validation Accuracy: 0.859375
Epoch  7, Batch 193 - Loss:   389.8320 Validation Accuracy: 0.859375
Epoch  7, Batch 194 - Loss:   392.8090 Validation Accuracy: 0.863281
Epoch  7, Batch 195 - Loss:   324.3616 Validation Accuracy: 0.863281
Epoch  7, Batch 196 - Loss:   367.5915 Validation Accuracy: 0.867188
Epoch  7, Batch 197 - Loss:   174.

Epoch  7, Batch 305 - Loss:   193.2054 Validation Accuracy: 0.867188
Epoch  7, Batch 306 - Loss:   383.0855 Validation Accuracy: 0.867188
Epoch  7, Batch 307 - Loss:   433.2640 Validation Accuracy: 0.867188
Epoch  7, Batch 308 - Loss:   224.0083 Validation Accuracy: 0.867188
Epoch  7, Batch 309 - Loss:   501.0132 Validation Accuracy: 0.863281
Epoch  7, Batch 310 - Loss:   235.9217 Validation Accuracy: 0.859375
Epoch  7, Batch 311 - Loss:   435.5192 Validation Accuracy: 0.863281
Epoch  7, Batch 312 - Loss:   425.4523 Validation Accuracy: 0.863281
Epoch  7, Batch 313 - Loss:   539.6108 Validation Accuracy: 0.863281
Epoch  7, Batch 314 - Loss:   171.5489 Validation Accuracy: 0.863281
Epoch  7, Batch 315 - Loss:   242.7669 Validation Accuracy: 0.863281
Epoch  7, Batch 316 - Loss:   463.6273 Validation Accuracy: 0.863281
Epoch  7, Batch 317 - Loss:   319.4608 Validation Accuracy: 0.863281
Epoch  7, Batch 318 - Loss:   236.6136 Validation Accuracy: 0.863281
Epoch  7, Batch 319 - Loss:   279.

Epoch  7, Batch 427 - Loss:   229.3640 Validation Accuracy: 0.871094
Epoch  7, Batch 428 - Loss:   274.7906 Validation Accuracy: 0.871094
Epoch  7, Batch 429 - Loss:   475.4321 Validation Accuracy: 0.871094
Epoch  8, Batch   1 - Loss:   299.2930 Validation Accuracy: 0.871094
Epoch  8, Batch   2 - Loss:   247.9175 Validation Accuracy: 0.871094
Epoch  8, Batch   3 - Loss:   349.6197 Validation Accuracy: 0.871094
Epoch  8, Batch   4 - Loss:   319.0555 Validation Accuracy: 0.871094
Epoch  8, Batch   5 - Loss:   341.7469 Validation Accuracy: 0.875000
Epoch  8, Batch   6 - Loss:   295.1556 Validation Accuracy: 0.871094
Epoch  8, Batch   7 - Loss:   256.6842 Validation Accuracy: 0.871094
Epoch  8, Batch   8 - Loss:   434.6677 Validation Accuracy: 0.871094
Epoch  8, Batch   9 - Loss:   563.1286 Validation Accuracy: 0.871094
Epoch  8, Batch  10 - Loss:   348.8524 Validation Accuracy: 0.871094
Epoch  8, Batch  11 - Loss:   480.3866 Validation Accuracy: 0.875000
Epoch  8, Batch  12 - Loss:   594.